In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

/Users/lucaspecina/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
'''PRESUPUESTOS'''


data = pd.read_csv('https://raw.githubusercontent.com/lucaspecina/presupuesto-consejomagis/master/presupuesto%20consejo%20magistratura%20-%20Sheet2.csv').iloc[:16]
data_concepto = data[['año','tipo presupuesto','GASTOS EN PERSONAL','BIENES DE CONSUMO',
                     'SERVICIOS NO PERSONALES','BIENES DE USO','TOTAL']]

# saco los cambios relativos por año (teniendo en cuenta el tipo de presupuesto)
data_pct_techos = pd.concat([data_concepto[data_concepto['tipo presupuesto']=='techos'][['año','tipo presupuesto']].sort_values('año',ascending=True).reset_index(drop=True),
                    data_concepto[data_concepto['tipo presupuesto'] =='techos'].sort_values('año',ascending=True).reset_index(drop=True).drop(columns=['año','tipo presupuesto']).pct_change().fillna(0)*100],axis='columns')
data_pct_ejecutado = pd.concat([data_concepto[data_concepto['tipo presupuesto']=='ejecutado'][['año','tipo presupuesto']].sort_values('año',ascending=True).reset_index(drop=True),
                    data_concepto[data_concepto['tipo presupuesto'] =='ejecutado'].sort_values('año',ascending=True).reset_index(drop=True).drop(columns=['año','tipo presupuesto']).pct_change().fillna(0)*100],axis='columns')
data_pct = pd.concat([data_pct_techos,data_pct_ejecutado])

# hago lo acumulado
data_pct_techos = pd.concat([data_concepto[data_concepto['tipo presupuesto']=='techos'][['año','tipo presupuesto']].sort_values('año',ascending=True).reset_index(drop=True),
                    data_concepto[data_concepto['tipo presupuesto'] =='techos'].sort_values('año',ascending=True).reset_index(drop=True).drop(columns=['año','tipo presupuesto']).pct_change().cumsum().fillna(0)*100],axis='columns')
data_pct_ejecutado = pd.concat([data_concepto[data_concepto['tipo presupuesto']=='ejecutado'][['año','tipo presupuesto']].sort_values('año',ascending=True).reset_index(drop=True),
                    data_concepto[data_concepto['tipo presupuesto'] =='ejecutado'].sort_values('año',ascending=True).reset_index(drop=True).drop(columns=['año','tipo presupuesto']).pct_change().cumsum().fillna(0)*100],axis='columns')
data_pct_acum = pd.concat([data_pct_techos,data_pct_ejecutado])

In [3]:
'''INFLACION'''

inflacion_anual =  pd.DataFrame({'año':[2013,2014,2015,2016,2017,2018,2019],'inflacion':[0,32.6,24.5,39.7,25.0,45.5,53.4]})
inflacion_anual['inflacion_acum'] = inflacion_anual.inflacion.cumsum()

inflacion_anual_2012 = pd.DataFrame({'año':[2012,2013,2014,2015,2016,2017,2018,2019],'inflacion':[0,27.3,32.6,24.5,39.7,25.0,45.5,53.4]})
inflacion_anual_2012['inflacion_acum'] = inflacion_anual_2012.inflacion.cumsum()

## Comparación del presupuesto del Consejo de la Magistratura con la inflación

**Tipo presupuesto**: 
- Según techos informados
- Ejecutado

**Concepto**:
- Gastos en personal
- Bienes de consumo
- Servicios no personales
- Bienes de uso

### Gráfico comparativo presupuesto vs inflación

In [4]:
def plotear(tipo_presupuesto,concepto):

    data_plot = data_pct[(data_pct['tipo presupuesto']==tipo_presupuesto)]
    data_plot_acum = data_pct_acum[(data_pct_acum['tipo presupuesto']==tipo_presupuesto)]
    
    data_inflacion = inflacion_anual if tipo_presupuesto=='ejecutado' else inflacion_anual_2012

    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.12,
                        subplot_titles=("Cambios porcentuales %", 'Cambios porcentuales acumulados %'))
    
    # PLOT CAMBIOS RELATIVOS    
    fig.add_trace(go.Scatter(x=data_inflacion['año'], y=data_inflacion['inflacion'],
                marker_color='black',name='inflacion'),row=1,col=1)

    fig.add_trace(go.Scatter(x=data_plot['año'], y=data_plot[concepto],
                marker_color='red',name='presupuesto'),row=1,col=1)
    
    # PLOT CAMBIOS ACUMULADOS
    fig.add_trace(go.Scatter(x=data_inflacion['año'], y=data_inflacion['inflacion_acum'],
                marker_color='black',name='inflacion',showlegend=False),row=2,col=1)
    
    fig.add_trace(go.Scatter(x=data_plot_acum['año'], y=data_plot_acum[concepto],
                marker_color='red',name='presupuesto',showlegend=False),row=2,col=1)
    
    # Update xaxis properties
    fig.update_xaxes(title_text="Año", row=1, col=1)
    fig.update_xaxes(title_text="Año", row=2,col=1)
    # Update yaxis properties
    fig.update_yaxes(title_text="porcentaje de cambio (%)", row=1, col=1)
    fig.update_yaxes(title_text="porcentaje de cambio acumulado (%)", row=2,col=1)
    
    fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))

    fig.update_layout(height=900, width=1000, title_text=f"Presupuesto ({tipo_presupuesto}) para {concepto} vs inflación")
    fig.show()
    
slider = widgets.interact(plotear, tipo_presupuesto=['techos','ejecutado'],
                concepto = ['TOTAL','GASTOS EN PERSONAL','BIENES DE CONSUMO','SERVICIOS NO PERSONALES','BIENES DE USO'])
display(slider)

interactive(children=(Dropdown(description='tipo_presupuesto', options=('techos', 'ejecutado'), value='techos'…

<function __main__.plotear(tipo_presupuesto, concepto)>

#### Presupuestos (original)

In [5]:
data_concepto.sort_values('año')

,año,tipo presupuesto,GASTOS EN PERSONAL,BIENES DE CONSUMO,SERVICIOS NO PERSONALES,BIENES DE USO,TOTAL
15,2012,techos,6947700,156800,2552000,422000,10078500
13,2013,techos,7415700,156800,2552000,422000,10546500
14,2013,ejecutado,8401581,174540,2999836,242283,11818240
11,2014,techos,8845000,181000,3307000,422000,12755000
12,2014,ejecutado,12333100,171995,3169540,94740,15769375
9,2015,techos,11643000,274000,4871000,422000,17210000
10,2015,ejecutado,17475696,243915,4684773,93945,22498329
7,2016,techos,18513000,343000,5512000,422000,24790000
8,2016,ejecutado,23086700,321810,5511237,244529,29164276
5,2017,techos,25257000,469900,8744000,1350400,35821300


#### Cambios porcentuales en los presupuestos

In [6]:
data_pct.sort_values('año')

,año,tipo presupuesto,GASTOS EN PERSONAL,BIENES DE CONSUMO,SERVICIOS NO PERSONALES,BIENES DE USO,TOTAL
0,2012,techos,0.000000,0.000000,0.000000,0.000000,0.000000
1,2013,techos,6.736042,0.000000,0.000000,0.000000,4.643548
0,2013,ejecutado,0.000000,0.000000,0.000000,0.000000,0.000000
2,2014,techos,19.273973,15.433673,29.584639,0.000000,20.940596
1,2014,ejecutado,46.794990,-1.458118,5.657109,-60.896968,33.432516
3,2015,techos,31.633691,51.381215,47.293620,0.000000,34.927479
2,2015,ejecutado,41.697513,41.815169,47.806085,-0.839139,42.671025
4,2016,techos,59.005411,25.182482,13.159515,0.000000,44.044160
3,2016,ejecutado,32.107471,31.935305,17.641495,160.289531,29.628632
5,2017,techos,36.428456,36.997085,58.635704,220.000000,44.498992


#### Cambios porcentuales en los presupuestos (acumulados)

In [7]:
data_pct_acum.sort_values('año')

,año,tipo presupuesto,GASTOS EN PERSONAL,BIENES DE CONSUMO,SERVICIOS NO PERSONALES,BIENES DE USO,TOTAL
0,2012,techos,0.000000,0.000000,0.000000,0.000000,0.000000
1,2013,techos,6.736042,0.000000,0.000000,0.000000,4.643548
0,2013,ejecutado,0.000000,0.000000,0.000000,0.000000,0.000000
2,2014,techos,26.010015,15.433673,29.584639,0.000000,25.584145
1,2014,ejecutado,46.794990,-1.458118,5.657109,-60.896968,33.432516
3,2015,techos,57.643706,66.814889,76.878259,0.000000,60.511624
2,2015,ejecutado,88.492503,40.357051,53.463195,-61.736106,76.103542
4,2016,techos,116.649117,91.997371,90.037775,0.000000,104.555784
3,2016,ejecutado,120.599974,72.292356,71.104690,98.553425,105.732173
5,2017,techos,153.077573,128.994455,148.673479,220.000000,149.054776


#### Inflación (por año y acumulado)

In [8]:
inflacion_anual_2012

,año,inflacion,inflacion_acum
0,2012,0.0,0.0
1,2013,27.3,27.3
2,2014,32.6,59.9
3,2015,24.5,84.4
4,2016,39.7,124.1
5,2017,25.0,149.1
6,2018,45.5,194.6
7,2019,53.4,248.0
